# Standard Least Squared
- No preprosessing(not even polynomials), plotting or anything fancy
- Plain Least Squared from exercise 3

###### Next steps: 

Low-fruit:
- make polynomials
- implement cross validation

High fruit:
- Maybe make a new document with only the Exploratory data analysis with PCA and such. Then we different algorithms can choose to highlight some result and use it in their Feature prosessing. (https://en.wikipedia.org/wiki/Exploratory_data_analysis)
- find or write code that finds the best degrees() can't plot...or can just plot one dimension at a time, better to just use ridge regression at that point? Does LS have any type of overfitting add-on

### Libraries and imports

In [2]:
# standard libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# own functions
from proj1_helpers import * 
import basic_functions as bf
import cross_validation as CV
#import plot_functions as pf 

#constants
train_path = 'train.csv'
test_path = 'test.csv'

### Importing data

In [3]:
y, x, ids = load_csv_data(train_path, sub_sample=False) #remember to switch of subsample when running it "for real"


print("y mean: ", np.mean(y))

y mean:  -0.314664


In [4]:

data = bf.clean_data(x, replace_no_measure_with_mean=False, replace_no_measure_with_median=True )

data 

array([[  1.38470000e+02,   5.16550000e+01,   9.78270000e+01, ...,
          1.24000000e+00,  -2.47500000e+00,   1.13497000e+02],
       [  1.60937000e+02,   6.87680000e+01,   1.03235000e+02, ...,
         -1.00000000e-02,  -2.00000000e-03,   4.62260000e+01],
       [  1.12406000e+02,   1.62172000e+02,   1.25953000e+02, ...,
         -1.00000000e-02,  -2.00000000e-03,   4.42510000e+01],
       ..., 
       [  1.05457000e+02,   6.05260000e+01,   7.58390000e+01, ...,
         -9.99000000e+02,  -9.99000000e+02,   4.19920000e+01],
       [  9.49510000e+01,   1.93620000e+01,   6.88120000e+01, ...,
         -9.99000000e+02,  -9.99000000e+02,   0.00000000e+00],
       [ -9.99000000e+02,   7.27560000e+01,   7.08310000e+01, ...,
         -9.99000000e+02,  -9.99000000e+02,   0.00000000e+00]])

In [5]:
data=bf.normalize(data)

data


array([[ 0.51914948,  0.07485418,  0.06812844, ...,  0.99675137,
         0.99439501,  0.06948372],
       [ 0.52940358,  0.09965294,  0.07215518, ...,  0.99550573,
         0.99686272,  0.02829991],
       [ 0.50725368,  0.23500634,  0.08907077, ...,  0.99550573,
         0.99686272,  0.0270908 ],
       ..., 
       [ 0.50408211,  0.08770931,  0.05175641, ...,  0.        ,
         0.        ,  0.02570782],
       [ 0.49928709,  0.02805782,  0.04652418, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.10543202,  0.04802751, ...,  0.        ,
         0.        ,  0.        ]])

In [6]:
data = bf.inverse_log_non_neg(data)

data


array([[-0.41815062, -0.07218501, -0.065908  , ...,  0.99675137,
         0.99439501,  0.06948372],
       [-0.42487784, -0.09499462, -0.06967081, ...,  0.99550573,
         0.99686272,  0.02829991],
       [-0.41028924, -0.2110761 , -0.08532483, ...,  0.99550573,
         0.99686272,  0.0270908 ],
       ..., 
       [-0.40818282, -0.08407393, -0.05046154, ...,  0.        ,
         0.        ,  0.02570782],
       [-0.40498972, -0.02767141, -0.04547437, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        , -0.10023623, -0.04690983, ...,  0.        ,
         0.        ,  0.        ]])

In [7]:
data = np.delete(data, [5, 6, 12, 24,25,26,27,28,29], axis=1) 

data = np.c_[np.ones((data.shape[0], 1)), data]

data

array([[ 1.        , -0.41815062, -0.07218501, ..., -0.11611478,
         0.66666667, -0.40755407],
       [ 1.        , -0.42487784, -0.09499462, ..., -0.07306616,
         0.33333333, -0.40087489],
       [ 1.        , -0.41028924, -0.2110761 , ..., -0.11686651,
         0.33333333, -0.40025065],
       ..., 
       [ 1.        , -0.40818282, -0.08407393, ..., -0.08898656,
         0.33333333, -0.39953616],
       [ 1.        , -0.40498972, -0.02767141, ..., -0.04856289,
         0.        ,  0.        ],
       [ 1.        ,  0.        , -0.10023623, ..., -0.04217063,
         0.        ,  0.        ]])

### Exploratory data analysis (ex: PCA)
https://en.wikipedia.org/wiki/Exploratory_data_analysis

### Feature Processing (ex:Fourier, Poly)
http://machinelearningmastery.com/discover-feature-engineering-how-to- engineer-features-and-how-to-get-good-at-it/.

Ideas:
- standardize
- polynomials
- fourier transformation
- sine, cos, tan ...ect
- look up kvantemechanics formulas an see if we can find some correlations

In [8]:
#initial_w = np.zeros(len(x[1]))
#max_iters = 100 # low
#gamma = 0.1
#tx = np.transpose(x) 
#ty = np.transpose(y)

#losses, w = bf.least_squares(ty, x)

#print(w.shape)


### Optimalizing: determining overfit or underfit
http://cs229.stanford.edu/materials/ML-advice.pdf

### Train and produce weigths 

### Visualize, find pattern of wrong predictions -> back to Exploratory/Feature Proc
http://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf

### Cross validation MAGIC 


In [9]:

#PROCESSING TEST DATA

pred_y, pred_x, pred_ids = load_csv_data(test_path, sub_sample=False)

pred_x = bf.clean_data(pred_x, replace_no_measure_with_mean=False, replace_no_measure_with_median=True )
pred_x = bf.normalize(pred_x)
#pred_x = bf.inverse_log_non_neg(pred_x)
pred_x = np.delete(pred_x, [5, 6, 12, 24,25,26,27,28,29], axis=1) 
pred_x = np.c_[np.ones((pred_x.shape[0], 1)), pred_x]

print("pred y shape:", pred_y.shape)
print("pred x shape: ", pred_x.shape)

y = bf.classify(y)

pred y shape: (568238,)
pred x shape:  (568238, 22)


In [10]:
copy_of_data = np.copy(data)
copy_of_pred_x = np.copy(pred_x)

copy_of_data



array([[ 1.        , -0.41815062, -0.07218501, ..., -0.11611478,
         0.66666667, -0.40755407],
       [ 1.        , -0.42487784, -0.09499462, ..., -0.07306616,
         0.33333333, -0.40087489],
       [ 1.        , -0.41028924, -0.2110761 , ..., -0.11686651,
         0.33333333, -0.40025065],
       ..., 
       [ 1.        , -0.40818282, -0.08407393, ..., -0.08898656,
         0.33333333, -0.39953616],
       [ 1.        , -0.40498972, -0.02767141, ..., -0.04856289,
         0.        ,  0.        ],
       [ 1.        ,  0.        , -0.10023623, ..., -0.04217063,
         0.        ,  0.        ]])

In [11]:

#PARAMATERS AND INITIAL WEIGHTS
max_iters = 5
gamma = 0.00065
initial_w=np.zeros((copy_of_data.shape[1], 1))
k_folds = 2

# RUNNING CROSS VALIDATION
avg_loss, losses, avg_preds, pred_acc_percents, best_weights = CV.cross_validation(bf.logistic_regression, y, copy_of_data, k_folds, initial_w, max_iters, gamma)

print("avg pred: ", avg_preds)
print("weights: ", best_weights)


#PREDICTING FOR KEGGLE USING BEST WEIGHTS FROM CROSS VALIDATION

predictions_for_keggle = bf.log_pred(copy_of_pred_x, best_weights)

#predicts = bf.sigmoid(np.dot(copy_of_pred_x,best_weights))

print("predicts for keggle: ","\n",predictions_for_keggle,"\n")



#INVERSING KEGGLE PREDICTIONS
predictions_for_keggle = predictions_for_keggle * -1

#print(predictions_for_keggle[0:25])
#print("mean predictions for keggle: ", np.mean(predictions_for_keggle))
#print("shape predictions for keggle: ", predictions_for_keggle.shape)


#CREATING FILE
#name = 'final.csv'
#create_csv_submission(pred_ids, predictions_for_keggle, name)


#lambda_=0.5
#initial_w = np.zeros((ready_x.shape[1], 1))
#max_iters = 50
#gamma = 0.0001
#k_folds = 3
#avg_loss, losses, avg_preds, pred_acc_percents = cross_validation(bf.reg_logistic_regression, y, ready_x, k_folds, lambda_, initial_w, max_iters, gamma)


#print("avg loss: ", avg_loss)
#print("Procentage wrong: ", avg_preds)


avg pred:  0.657332
weights:  [[ 17.10470404]
 [ -7.15211482]
 [  2.04931855]
 [ -0.88299597]
 [ -1.13874047]
 [-11.81649728]
 [ -6.07061357]
 [ -0.09310684]
 [ -2.75218651]
 [  0.34632361]
 [ 29.9968238 ]
 [ -1.71571377]
 [  8.41613291]
 [  8.45133701]
 [ -0.38939749]
 [  8.61404172]
 [  8.71878553]
 [ -0.29382166]
 [  8.92625301]
 [ -3.08683431]
 [ 14.02934544]
 [ -7.34165301]]
predicts for keggle:  
 [ 1.  1.  1. ...,  1.  1.  1.] 



In [ ]:
def build_k_indices(y, k_fold, seed=1):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

def split_k(x,y,k_indices, k):
    #Given the data x, y, k_indices build by build_k_idiices, and k
    #Splits the data into training and test data, where the k-th fold is the test data
    x_test, y_test= x[k_indices[k]], y[k_indices[k]]
    train_ind=np.delete(k_indices,k,0)
    train_ind=np.ravel(train_ind)
    x_tr, y_tr= x[train_ind], y[train_ind]
    return x_test, y_test, x_tr, y_tr 




### Predict higgs boson

In [ ]:
# Import test.csv which contains nonlabeled data
# Remember to have sub_sample=False when submitting to kaggle WARNING: it takes a looooong time
pred_y, pred_x, pred_ids = load_csv_data(test_path, sub_sample=False)
print(pred_x, pred_y)
print(pred_ids)

In [ ]:
weights = w

#predictions = compute_prediction(pred_x, weights)
#print(predictions[1:25])

y_predictions = predict_labels(weights, pred_x)
print(y_predictions[1:25])


In [ ]:
# creates a file with the name you want in the folder of the Python-file. just look in the folder :) 
#name = 'powerpuff_testing.csv'

#print(y_predictions.shape)
#print(pred_ids.shape)

#create_csv_submission(pred_ids, y_predictions, name)

#print("yolo")

#
# THE END
#